In [1]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import get_cosine_schedule_with_warmup

sys.path.append("../../run/")
sys.path.append("../../")
from experimet_config import Config
from src.utils.utils import set_seed
from src.datasets.latent_navigation_dataset import KeywordNavigationDataset
from src.models.regressor import Regressor
from src.models.latent_walker import WalkEffKANMulti,WalkEffKAN,WalkMlpMultiW
from src.models.deepsdf import DeepSDF
from src.utils.metric import regressor_criterion

In [2]:
set_seed(42)
cfg = Config()

# loading keyword columns
keyword_columns = cfg.keyword_attribute

# loading training data
merge_df = pd.read_csv("../../data/table/merged.csv").dropna().reset_index(drop=True)
merge_df = merge_df[~merge_df["folder_name"].isin(cfg.noise_data)]
for col in keyword_columns:
    # 0~1にmin-max scaling
    merge_df[col] = (merge_df[col] - merge_df[col].min()) / (
        merge_df[col].max() - merge_df[col].min()
    )

# loading carfolder2latent_idx
with open("../../outputs/data-split/carfolder2latent_idx.pickle", "rb") as f:
    carfolder2latent_idx = pickle.load(f)

In [3]:
# loading latent data
latent_codes = torch.load("../../outputs/latent-codes/latent_codes.pth")
latent_walker = WalkEffKANMulti(
    attribute_dim=len(keyword_columns), latent_code_dim=cfg.latent_code_dim
).to(cfg.device)

latent_walker.load_state_dict(torch.load("../../outputs/models/keyword_walker_kan.pth"))

deepsdf = DeepSDF(
    hidden_dim=cfg.hidden_dim,
    xyz_pos_enc_dim=cfg.xyz_pos_enc_dim,
    latent_code_dim=cfg.latent_code_dim,
).to(cfg.device)
deepsdf.load_state_dict(torch.load("../../outputs/models/deepsdf.pth"))

<All keys matched successfully>

In [4]:
edit_car_info = merge_df.iloc[0]
edit_car_info

id                                                                        4
maker                                                                  Audi
car_name                                                          Q4 e-tron
year                                                                   2021
engine                                                                  BEV
country                                                                 ドイツ
mode_url                  https://hum3d.com/ja/3d-models/audi-q4-e-tron-...
file_name                                        Audi_Q4_e-tron_S-line_2021
folder_name                                      004_AU_Q4_etron_Sline_2021
Voluminous_Smart                                                   0.346418
Powerful_Delicate                                                  0.349883
Linear_Curvy                                                       0.514678
Functional_Decorative                                              0.665962
Robust_Flexi

In [5]:
from src.utils.mesh_utils import create_mesh
from ipywidgets import interact_manual, FloatSlider


latent_code = (
    latent_codes[carfolder2latent_idx[edit_car_info["folder_name"]]]
    .reshape(1, -1)
    .to(cfg.device)
)

base_keyword = edit_car_info[keyword_columns].values


@interact_manual
def create_3d(
    lambda_=FloatSlider(value=3, min=0, max=6, step=1),
    ボリューム感のある_スマートな=FloatSlider(value=0, min=-1, max=1, step=0.1),
    力強い_繊細な=FloatSlider(value=0, min=-1, max=1, step=0.1),
    直線的な_曲線的な=FloatSlider(value=0, min=-1, max=1, step=0.1),
    機能的な_装飾的な=FloatSlider(value=0, min=-1, max=1, step=0.1),
    堅牢な_しなやかな=FloatSlider(value=0, min=-1, max=1, step=0.1),
    落ち着いた_躍動感のある=FloatSlider(value=0, min=-1, max=1, step=0.1),
    現実的な_ロマンのある=FloatSlider(value=0, min=-1, max=1, step=0.1),
    エレガントな_かわいらしい=FloatSlider(value=0, min=-1, max=1, step=0.1),
    大人っぽい_若々しい=FloatSlider(value=0, min=-1, max=1, step=0.1),
    高級そうな_手頃そうな=FloatSlider(value=0, min=-1, max=1, step=0.1),
    儀礼的な_普段使いの=FloatSlider(value=0, min=-1, max=1, step=0.1),
    厳格な_親しみやすい=FloatSlider(value=0, min=-1, max=1, step=0.1),
    画一的な_自由な=FloatSlider(value=0, min=-1, max=1, step=0.1),
    特別な_日常的な=FloatSlider(value=0, min=-1, max=1, step=0.1),
):
    # main-process

    delta_keyword = np.array(
        [
            ボリューム感のある_スマートな,
            力強い_繊細な,
            直線的な_曲線的な,
            機能的な_装飾的な,
            堅牢な_しなやかな,
            落ち着いた_躍動感のある,
            現実的な_ロマンのある,
            エレガントな_かわいらしい,
            大人っぽい_若々しい,
            高級そうな_手頃そうな,
            儀礼的な_普段使いの,
            厳格な_親しみやすい,
            画一的な_自由な,
            特別な_日常的な,
        ]
    )

    edit_keyword = base_keyword + delta_keyword
    edit_keyword = np.clip(edit_keyword, 0, 1)
    delta = edit_keyword - base_keyword



    changed_attributes = []
    for i, change in enumerate(delta):
        if change != 0:
            changed_attributes.append(f"{keyword_columns[i]}_{base_keyword[i]:.3f}_{edit_keyword[i]:.3f}")

    model_name = "A4_keyword_kan"
    folder_name = model_name + "_" + "_".join(changed_attributes)
    os.makedirs(folder_name, exist_ok=True)  # 创建子文件夹

    print("ボリューム感のある-スマートな: ", base_keyword[0], " => ", edit_keyword[0])
    print("力強い-繊細な: ", base_keyword[1], " => ", edit_keyword[1])
    print("直線的な-曲線的な: ", base_keyword[2], " => ", edit_keyword[2])
    print("機能的な-装飾的な: ", base_keyword[3], " => ", edit_keyword[3])
    print("堅牢な-しなやかな: ", base_keyword[4], " => ", edit_keyword[4])
    print("落ち着いた-躍動感のある: ", base_keyword[5], " => ", edit_keyword[5])
    print("現実的な-ロマンのある: ", base_keyword[6], " => ", edit_keyword[6])
    print("エレガントな-かわいらしい: ", base_keyword[7], " => ", edit_keyword[7])
    print("大人っぽい-若々しい: ", base_keyword[8], " => ", edit_keyword[8])
    print("高級そうな-手頃そうな: ", base_keyword[9], " => ", edit_keyword[9])
    print("儀礼的な-普段使いの: ", base_keyword[10], " => ", edit_keyword[10])
    print("厳格な-親しみやすい: ", base_keyword[11], " => ", edit_keyword[11])
    print("画一的な-自由な: ", base_keyword[12], " => ", edit_keyword[12])
    print("特別な-日常的な: ", base_keyword[13], " => ", edit_keyword[13])

    n_divide = 2
    file_names = ["{}".format(i) for i in range(n_divide)]
    divided_delta = np.linspace(0, delta, n_divide)

    for i, delta in enumerate(divided_delta):
        delta = (
            torch.from_numpy(delta.astype(np.float32))
            .float()
            .to(cfg.device)
            .reshape(1, -1)
        )
        edited_latent = latent_walker(
            latent_code, delta.to(cfg.device), lambda_=lambda_
        )
        edited_latent = torch.tensor(edited_latent).float().to(cfg.device)
        create_mesh(
            deepsdf,
            edited_latent,
            "{}/{}".format(folder_name, file_names[i]),  # Updated path,
            N=512,
            max_batch=80**3,
            offset=None,
            scale=None,
        )

    # return-process
    return "created 3d model!"

interactive(children=(FloatSlider(value=3.0, description='lambda_', max=6.0, step=1.0), FloatSlider(value=0.0,…